In [1]:
import requests 
import pandas as pd 
import numpy as np 

api_key = 'e22c368541904516c60dfa05148ef976'

stock = 'AAPL'
stockprices = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{stock}?serietype=line&apikey={api_key}').json()

stockprices = stockprices['historical'][0:1200]

stockprices = pd.DataFrame.from_dict(stockprices)
stockprices = stockprices.set_index('date')

stockprices = stockprices.iloc[::-1]

stockprices['20d'] = stockprices['close'].rolling(20).mean() 


stockprices['return'] = np.log(stockprices['close'] / stockprices['close'].shift(1) )


stockprices['difference'] = stockprices['close'] - stockprices['20d']

stockprices['long'] = np.where(stockprices['difference'] < -2 ,1,np.nan)
stockprices['long'] = np.where(stockprices['difference'] * stockprices['difference'].shift(1) < 0, 0, stockprices['long'])
stockprices['long'] = stockprices['long'].ffill().fillna(0)

stockprices['gain_loss'] = stockprices['long'].shift(1) * stockprices['return']
stockprices = stockprices.dropna(subset=['20d'])

stockprices['total'] =  stockprices['gain_loss'].cumsum()
print(stockprices.tail(30))

                 close         20d    return  ...  long  gain_loss     total
date                                          ...                           
2021-09-30  141.500000  148.096000 -0.009355  ...   1.0  -0.009355 -0.143274
2021-10-01  142.649994  147.546000  0.008094  ...   1.0   0.008094 -0.135180
2021-10-04  139.139999  146.788000 -0.024913  ...   1.0  -0.024913 -0.160094
2021-10-05  141.110001  146.009000  0.014059  ...   1.0   0.014059 -0.146034
2021-10-06  142.000000  145.353500  0.006287  ...   1.0   0.006287 -0.139747
2021-10-07  143.289993  144.814499  0.009043  ...   1.0   0.009043 -0.130704
2021-10-08  142.899994  144.510999 -0.002725  ...   1.0  -0.002725 -0.133429
2021-10-11  142.809998  144.173998 -0.000630  ...   1.0  -0.000630 -0.134059
2021-10-12  141.509995  143.843498 -0.009145  ...   1.0  -0.009145 -0.143204
2021-10-13  140.910004  143.437499 -0.004249  ...   1.0  -0.004249 -0.147453
2021-10-14  143.759995  143.185999  0.020024  ...   0.0   0.020024 -0.127429

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
